In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sys

#instant execution, useful for dev
tf.enable_eager_execution()

print(tf.__version__)
print(tf.keras.__version__)
print(pd.__version__)
print(np.__version__)
print (sys.version_info)

1.14.0
2.2.4-tf
0.24.2
1.16.4
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)


In [5]:
#load data
from google.colab import drive
drive.mount('/content/drive/')

dftrain=pd.read_csv('/content/drive/My Drive/train.csv')
dfpredict=pd.read_csv('/content/drive/My Drive/test.csv')

print(dftrain.head())
print(dftrain.shape[0])
print(dfpredict.head())
print(dfpredict.shape[0])

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
   Store  Dept        Date  ...  Store2  Dept2  Weekly_Sales
0      1     1  2010-02-05  ...     0.0    0.0      24924.50
1      1     1  2010-02-12  ...     0.0    0.0      46039.49
2      1     1  2010-02-19  ...     0.0    0.0      41595.55
3      1     1  2010-02-26  ...     0.0    0.0      19403.54
4      1     1  2010-03-05  ...     0.0    0.0      21827.90

[5 rows x 16 columns]
421570
   Store  Dept        Date  ...  IsHoliday_True  Store2  Dept2
0      1     1  2012-11-02  ...               0     0.0    0.0
1      1     1  2012-11-09  ...               0     0.0    0.0
2      1     1  2012-11-16  ...               0     0.0    0.0
3      1     1  2012-11-23  ...               1     0.0    0.0
4      1     1  2012-11-30  ...               0     0.0    0.0

[5 rows x 15 columns]
115064


In [6]:
#I don't need the Date,Store and Dept columns for NN (but used for submission)
dftrain.drop("Date", axis=1, inplace=True) 
dftrain.drop("Store", axis=1, inplace=True) 
dftrain.drop("Dept", axis=1, inplace=True) 
dfpredict.drop("Date", axis=1, inplace=True) 
dfpredict.drop("Store", axis=1, inplace=True) 
dfpredict.drop("Dept", axis=1, inplace=True) 

print(dftrain.shape[0])
print(dfpredict.shape[0])

421570
115064


In [7]:
#remove most features
dftrain.drop("Size", axis=1, inplace=True) 
dftrain.drop("Type_A", axis=1, inplace=True) 
dftrain.drop("Type_B", axis=1, inplace=True) 
dftrain.drop("Type_C", axis=1, inplace=True) 
dftrain.drop("Temperature", axis=1, inplace=True) 
dftrain.drop("Fuel_Price", axis=1, inplace=True) 
dftrain.drop("IsHoliday_False", axis=1, inplace=True) 
dftrain.drop("IsHoliday_True", axis=1, inplace=True) 

dfpredict.drop("Size", axis=1, inplace=True) 
dfpredict.drop("Type_A", axis=1, inplace=True) 
dfpredict.drop("Type_B", axis=1, inplace=True) 
dfpredict.drop("Type_C", axis=1, inplace=True) 
dfpredict.drop("Temperature", axis=1, inplace=True) 
dfpredict.drop("Fuel_Price", axis=1, inplace=True) 
dfpredict.drop("IsHoliday_False", axis=1, inplace=True) 
dfpredict.drop("IsHoliday_True", axis=1, inplace=True) 

print(dftrain.head())
print(dftrain.shape[0])
print(dfpredict.head())
print(dfpredict.shape[0])

   Seasonality  Year  Store2  Dept2  Weekly_Sales
0     0.078431   0.0     0.0    0.0      24924.50
1     0.098039   0.0     0.0    0.0      46039.49
2     0.117647   0.0     0.0    0.0      41595.55
3     0.137255   0.0     0.0    0.0      19403.54
4     0.156863   0.0     0.0    0.0      21827.90
421570
   Seasonality      Year  Store2  Dept2
0     0.843137  0.666667     0.0    0.0
1     0.862745  0.666667     0.0    0.0
2     0.882353  0.666667     0.0    0.0
3     0.901961  0.666667     0.0    0.0
4     0.921569  0.666667     0.0    0.0
115064


In [0]:
#force all types to float32
dftrain = dftrain.astype(np.float32)
dfpredict = dfpredict.astype(np.float32)

#split train data in features and labels
y = dftrain.Weekly_Sales
x = dftrain.drop('Weekly_Sales', axis=1)

#split in train (80%) and test (20%) sets 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2)

In [0]:
#convert the dataframes to numpy arrays
#y_train = y_train.to_numpy() requires 0.24
y_train = y_train.values
x_train = x_train.values
y_test = y_test.values
x_test = x_test.values
x_predict = dfpredict.values

In [0]:
#here is the NN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=4, activation='relu', kernel_initializer='normal', input_dim=4))
model.add(Dense(units=1, kernel_initializer='normal'))
model.compile(loss='mean_squared_error',
              optimizer='Adam',
              metrics=['accuracy'])

In [12]:
#train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=25)

# evaluate the model
_, accuracy = model.evaluate(x_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

Train on 337256 samples, validate on 84314 samples
Epoch 1/25
337256/337256 [==============================] - 13s 39us/sample - loss: 753021637.2664 - acc: 3.8843e-04 - val_loss: 723516447.6212 - val_acc: 4.2698e-04
Epoch 2/25
337256/337256 [==============================] - 12s 37us/sample - loss: 673177777.9895 - acc: 3.9139e-04 - val_loss: 626208936.6196 - val_acc: 4.2698e-04
Epoch 3/25
337256/337256 [==============================] - 12s 37us/sample - loss: 579537361.8498 - acc: 3.9139e-04 - val_loss: 546142039.2419 - val_acc: 4.2698e-04
Epoch 4/25
337256/337256 [==============================] - 13s 37us/sample - loss: 528244496.4699 - acc: 3.9139e-04 - val_loss: 522818210.2385 - val_acc: 4.2698e-04
Epoch 5/25
337256/337256 [==============================] - 12s 37us/sample - loss: 518974421.9424 - acc: 3.9139e-04 - val_loss: 519969370.5036 - val_acc: 4.2698e-04
Epoch 6/25
337256/337256 [==============================] - 12s 37us/sample - loss: 517073358.0224 - acc: 3.9139e-04 - 

In [0]:
# Predict!!
y_predict = model.predict(x_predict)

#for i in range(len(x_predict)):
for i in range(10):
	print("x=%s, y=%s" % (x_predict[i], y_predict[i]))

x=[0.84313726 0.6666667  0.6302673  1.         0.         0.
 0.5731417  0.45791584 1.         0.         0.         0.        ], y=[16678.842]
x=[0.8627451  0.6666667  0.6302673  1.         0.         0.
 0.6273343  0.42184368 1.         0.         0.         0.        ], y=[16742.732]
x=[0.88235295 0.6666667  0.6302673  1.         0.         0.
 0.5511717  0.39078155 1.         0.         0.         0.        ], y=[16669.807]
x=[0.9019608  0.6666667  0.6302673  1.         0.         0.
 0.581472   0.37024048 0.         1.         0.         0.        ], y=[19770.387]
x=[0.92156863 0.6666667  0.6302673  1.         0.         0.
 0.5458623  0.36823648 1.         0.         0.         0.        ], y=[16722.26]
x=[0.9411765  0.6666667  0.6302673  1.         0.         0.
 0.65369827 0.36372745 1.         0.         0.         0.        ], y=[16875.166]
x=[0.9607843 0.6666667 0.6302673 1.        0.        0.        0.5142805
 0.3486974 1.        0.        0.        0.       ], y=[16749.08

In [0]:
#save predictions to a csv file
from google.colab import drive
drive.mount('/content/drive/')
np.savetxt("/content/drive/My Drive/prediction.csv", y_predict, fmt="%10.5f", delimiter=",")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
